In [1]:
# Test ORPOTrainer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
from transformers import TrainingArguments
from trl import ORPOConfig, ORPOTrainer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

alpaca_prompt = """Below is a writing instruction. Follow the instruction to compose a coherent, engaging and high-quality narrative.

### Instruction:
{}

### Response:
{}"""

def format_prompt(sample):
    '''
    Use chat template to format the prompt and the generated tokens. 
    '''
    #if tokenizer.chat_template is None:

    sample['prompt_chosen'] = alpaca_prompt.format(sample['old_ins'].strip(), "")
    sample['prompt_rejected'] = alpaca_prompt.format(sample['new_ins'].strip(), "")
    sample["answer"] = sample['processed_text'].strip() + "</s>"
    return sample
pass


dataset = load_dataset("chtmp223/suri", split="train[:1]")
dataset = dataset.map(format_prompt,)
print(len(dataset[0]['prompt_chosen']), len(dataset[0]['prompt_rejected']), len(dataset[0]['answer']))
max_seq_length = 1024

orpo_trainer = ORPOTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    args = ORPOConfig(
        max_length = max_seq_length,
        max_prompt_length = max_seq_length//2,
        max_completion_length = max_seq_length//2,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        beta = 0.1,
        logging_steps = 1,
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        max_steps = 30, # Change to num_train_epochs = 1 for full training runs
        output_dir = "outputs",
    ),
)

orpo_trainer.train()

/Users/ctpham/anaconda3/envs/orpo/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


2147 2147 26930


/Users/ctpham/Downloads/trl/trl/trainer/orpo_trainer.py:246: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/Users/ctpham/anaconda3/envs/orpo/lib/python3.11/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
/Users/ctpham/anaconda3/envs/orpo/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of b

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


wandb: Currently logged in as: chtmp223. Use `wandb login --relogin` to force relogin


  0%|          | 0/30 [00:00<?, ?it/s]

tensor([[ -30.8675,  -28.5200,  -33.2322,  ...,  -39.9825,  -37.0612,
          -25.6073],
        [ -43.5112,  -41.7747,  -48.6887,  ...,  -52.4905,  -48.7237,
          -39.6207],
        [ -46.4937,  -44.9711,  -49.7434,  ...,  -51.5181,  -49.7477,
          -40.7849],
        ...,
        [ -29.9010,  -31.1083,  -36.7706,  ...,  -42.2621,  -35.3417,
          -24.8276],
        [ -46.8705,  -45.9142,  -49.7150,  ...,  -53.8008,  -49.4960,
          -38.6803],
        [-190.9165, -187.1347, -193.5081,  ..., -208.4265, -208.9756,
         -181.3655]], device='mps:0', grad_fn=<SelectBackward0>)
CHOSEN PROMPT LOGP SHAPE: torch.Size([1])
REJECTED PROMPT LOGP SHAPE: torch.Size([1])


Could not estimate the number of tokens of the input, floating-point operations will not be computed


AssertionError: Torch not compiled with CUDA enabled